<a href="https://colab.research.google.com/github/euphoria96/KB_Smishing_dacon/blob/master/Model4_BiLSTM_ngram_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting
Mounting google drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# data preprocessing
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
# tokenizer
import re
#from konlpy.tag import Mecab
# modeling
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Flatten, Input, SpatialDropout1D, Conv1D, MaxPooling1D, GRU, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score
import pickle
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings(action='ignore')
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
random.randint(0,9999999)

4897923

In [0]:
sd = 4897923
np.random.seed(sd)
random.seed(sd)
os.environ['PYTHONHASHSEED']=str(sd)

from tensorflow.keras import backend as K
config = tf.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
tf.set_random_seed(sd)
sess = tf.Session(graph=tf.get_default_graph(), config=config)
K.set_session(sess)

### Load Data

In [5]:
cd /content/gdrive/My Drive/Colab Notebooks/smishing/

/content/gdrive/My Drive/Colab Notebooks/smishing


In [6]:
train = pd.read_csv('data/train_processed.csv')
test = pd.read_csv('data/test_processed.csv')
submission=pd.read_csv('data/submission_제출양식.csv')
print(train.shape, test.shape, submission.shape)
train.head(3)

(295945, 7) (1626, 6) (1626, 2)


,id,year_month,text,smishing,text_,2gram,3gram
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,은행 성산 팀장 입니다 행복 한 주말 되 세요,은행.성산 성산.팀장 팀장.입니다 입니다.행복 행복.한 한.주말 주말.되 되.세요,은행.성산.팀장 성산.팀장.입니다 팀장.입니다.행복 입니다.행복.한 행복.한.주말 ...
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,오늘 도 많이 웃 으시 는 하루 시작 하 세요 은행 진월동 vip 라운지 올림,오늘.도 도.많이 많이.웃 웃.으시 으시.는 는.하루 하루.시작 시작.하 하.세요 ...,오늘.도.많이 도.많이.웃 많이.웃.으시 웃.으시.는 으시.는.하루 는.하루.시작 ...
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,안녕 하 십니까 고객 님 은행 입니다 금일 납부 하 셔야 할 금액 은 원 입니다 감...,안녕.하 하.십니까 십니까.고객 고객.님 님.은행 은행.입니다 입니다.금일 금일.납...,안녕.하.십니까 하.십니까.고객 십니까.고객.님 고객.님.은행 님.은행.입니다 은행...


## Data Preparation


### Sampling
- Counter({0: 277242, 1: 18703}) #0.0631975535994864
- mixed sampling

In [0]:
def train_data_sampling(train, seed=1234, a=3, b=3):
    train_nsm_idx=list(train[train['smishing']==0].index)
    train_sm_idx=list(train[train['smishing']==1].index)
    random.seed(seed)
    train_nsm_idx = random.sample(train_nsm_idx, k=18703*a)
    random.seed(seed)
    train_sm_idx = random.choices(train_sm_idx, k=18703*b)
    train_idx = train_nsm_idx + train_sm_idx
    print(train_idx[:5])
    random.shuffle(train_idx)
    print(train_idx[:5])
    df_train = train.iloc[train_idx].reset_index(drop=True)
    return df_train

### text to sequences

In [0]:
def text2sequence(train_text, test_text, max_len=1000):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_text)
    train_X_seq = tokenizer.texts_to_sequences(train_text)
    vocab_size = len(tokenizer.word_index) + 1
    print('vocab size: ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen=max_len)
    test_X_seq = tokenizer.texts_to_sequences(test_text)
    X_test = pad_sequences(test_X_seq, maxlen=max_len)
    return X_train, X_test, vocab_size

### scoring for modeling

In [0]:
auc_=0
def auc_score(y_true, y_pred):
    global auc_
    try:
        auc_ = roc_auc_score( y_true, y_pred, average='macro', sample_weight=None).astype('float32')
    except ValueError:
        pass
    return auc_

def auc( y_true, y_pred ) :
    score = tf.py_func( lambda y_true, y_pred : auc_score(y_true, y_pred) , [y_true, y_pred], 'float32', stateful=False, name='sklearnAUC' )
    return score

In [0]:
def model_save(model, mname):
    model_json = model.to_json()
    with open('model/'+mname+'.json', 'w') as json_file : 
        json_file.write(model_json)
    model.save_weights('model/'+mname+'.h5')

## Preprocessing

In [12]:
x1 = pd.DataFrame(pd.Series([len(x.split()) for x in train['text_']]).describe(), columns=['text_']).transpose()
x2 = pd.DataFrame(pd.Series([len(x.split()) for x in train['2gram']]).describe(), columns=['2gram']).transpose()
x3 = pd.DataFrame(pd.Series([len(x.split()) for x in train['3gram']]).describe(), columns=['3gram']).transpose()
pd.concat([x1,x2,x3], axis=0)

,count,mean,std,min,25%,50%,75%,max
text_,295945.0,70.010012,90.041335,1.0,16.0,35.0,82.0,664.0
2gram,295945.0,69.010056,90.041308,1.0,15.0,34.0,81.0,663.0
3gram,295945.0,68.010191,90.041213,1.0,14.0,33.0,80.0,662.0


In [13]:
x1 = pd.DataFrame(pd.Series([len(x) for x in train['text_']]).describe(), columns=['text_']).transpose()
x2 = pd.DataFrame(pd.Series([len(x) for x in train['2gram']]).describe(), columns=['2gram']).transpose()
x3 = pd.DataFrame(pd.Series([len(x) for x in train['3gram']]).describe(), columns=['3gram']).transpose()
pd.concat([x1,x2,x3], axis=0)

,count,mean,std,min,25%,50%,75%,max
text_,295945.0,188.277727,241.091119,1.0,43.0,94.0,220.0,1660.0
2gram,295945.0,371.484614,482.234776,1.0,82.0,183.0,435.0,3315.0
3gram,295945.0,549.332724,723.312468,1.0,114.0,266.0,645.0,4965.0


In [15]:
df_train = train_data_sampling(train, seed=sd, a=5, b=3)
train_y = df_train['smishing']

train_X, test_X, vocab_size = text2sequence(df_train['text_'], test['text_'], max_len=600)
train_X2, test_X2, vocab_size2 = text2sequence(df_train['2gram'], test['2gram'], max_len=600)
train_X3, test_X3, vocab_size3 = text2sequence(df_train['3gram'], test['3gram'], max_len=600)
print(train_X.shape, train_y.shape)

[38042, 12810, 200714, 14770, 40659]
[110894, 135712, 85181, 231328, 176914]
vocab size:  26314
vocab size:  26314
vocab size:  26310
(149624, 600) (149624,)


## Modeling - BiLSTM

In [0]:
def BiLSTM(vocab_size, max_len=600):
    model = Sequential()
    model.add(Embedding(vocab_size, 64, input_length = max_len))
    model.add(SpatialDropout1D(0.5))
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='tanh', kernel_regularizer = regularizers.l2(0.001)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[auc])
    model.summary()
    return model

### 1. 1 gram

In [0]:
## single word
model1 = BiLSTM(vocab_size, max_len=600)
history = model1.fit(train_X, train_y, epochs=5, batch_size=128, validation_split=0.2)

mname = (datetime.now() + timedelta(hours=9) ).isoformat()[5:-10]+'_BiLSTM_600_ep5'
model_save(model1, mname)

y_pred = model1.predict(test_X, batch_size=128)
submission['smishing'] = y_pred
submission.to_csv('submission/'+mname+'.csv',index=False)

NameError: ignored

### 2. 2 gram

In [0]:
## 2 gram
model2 = BiLSTM(vocab_size2, max_len=600)
early_stopping = EarlyStopping(patience=2)
history = model2.fit(train_X2, train_y, epochs=50, batch_size=128, validation_split=0.3, callbacks=[early_stopping])

mname = (datetime.now() + timedelta(hours=9) ).isoformat()[5:-10]+'_BiLSTM_600_2gram_es'
model_save(model2, mname)

y_pred = model2.predict(test_X2, batch_size=128)
submission['smishing'] = y_pred
submission.to_csv('submission/'+mname+'.csv',index=False)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 64)           1684096   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 600, 64)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 1,758,465
Trainable params: 1,758,465
Non-trainable params: 0
____________________________________________